In [2]:
# This is an Agent Necessary Imports
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv

import asyncio
import sendgrid
import os

from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown


In [3]:
# Load environment variables (e.g., SENDGRID_API_KEY)
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [4]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name = "Search Agent",
    instructions=INSTRUCTIONS,
    tools = [WebSearchTool(search_context_size="high")],
    model = "gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)



In [6]:
message = "IS there a Word or sentence or Quote to remember everytime I facing that fear"

with trace("Search"):
    result = await Runner.run(search_agent,message)

display(Markdown(result.final_output))

Facing fear is a common challenge, and having a mantra can be empowering. Here are some quotes to inspire courage:

- "The only thing we have to fear is fear itself." — Franklin D. Roosevelt ([guideposts.org](https://guideposts.org/positive-living/health-and-wellness/life-advice/managing-life-changes/7-inspiring-quotes-to-overcome-fear/?utm_source=openai))

- "Courage is not the absence of fear, but the triumph over it." — Nelson Mandela ([guideposts.org](https://guideposts.org/positive-living/health-and-wellness/life-advice/managing-life-changes/7-inspiring-quotes-to-overcome-fear/?utm_source=openai))

- "Do the thing you fear, and the death of fear is certain." — Ralph Waldo Emerson ([quotesanity.com](https://quotesanity.com/quotes-to-help-you-conquer-your-fears/?utm_source=openai))

- "You gain strength, courage, and confidence by every experience in which you really stop to look fear in the face." — Eleanor Roosevelt ([quotesanity.com](https://quotesanity.com/quotes-to-help-you-conquer-your-fears/?utm_source=openai))

- "Fear is only as deep as the mind allows." — Japanese Proverb ([quotesanity.com](https://quotesanity.com/quotes-to-help-you-conquer-your-fears/?utm_source=openai))

These words can serve as reminders to confront and overcome fear. 

In [46]:
How_Many_Searches = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {How_Many_Searches} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)




In [47]:

message = (
    "I keep trying but I can't stay consistent. I'm afraid I'll fail again like before, "
    "even though deep down I know that's just fear talking, not reality. "
    "How do I face this fear and finally stay the course? "
    "Give me motivation, quotes to remember, and one thing I can do right now to break this pattern."
)


result = await Runner.run(planner_agent, message)



print(result.final_output)

searches=[WebSearchItem(reason='To provide motivational quotes that can help inspire consistency and courage.', query='motivational quotes for facing fears and staying consistent'), WebSearchItem(reason='To find guidance on overcoming fear and building consistency in habits.', query='how to overcome fear and maintain consistency'), WebSearchItem(reason='To explore strategies for breaking negative patterns and establishing positive habits.', query='strategies to break negative patterns and build good habits'), WebSearchItem(reason='To gather personal development techniques that help with motivation and facing fears.', query='personal development techniques for motivation and bravery'), WebSearchItem(reason='To find immediate actionable steps to take right now for breaking out of a pattern of inconsistency.', query='one actionable step to improve consistency and face fears')]


In [50]:
# === STEP 4: Get the Search Plan ===
print("📋 Creating search plan...")
plan_result = await Runner.run(planner_agent, message)
search_plan = plan_result.final_output

print(f"✅ Plan created with {len(search_plan.searches)} searches\n")


# === STEP 5: Execute Each Search ===
search_results = []

for i, search_item in enumerate(search_plan.searches, 1):
    print(f"🔍 Search {i}/{len(search_plan.searches)}: {search_item.query}")
    
    # Run the search agent
    result = await Runner.run(search_agent, search_item.query)
    
    search_results.append({
        "query": search_item.query,
        "reason": search_item.reason,
        "summary": result.final_output
    })
    
    print(f"✅ Completed\n")


📋 Creating search plan...
✅ Plan created with 5 searches

🔍 Search 1/5: motivational quotes about overcoming fear and staying consistent
✅ Completed

🔍 Search 2/5: how to face fear and stay consistent in habits
✅ Completed

🔍 Search 3/5: success stories overcoming fear of failure
✅ Completed

🔍 Search 4/5: mindfulness techniques for overcoming fear and anxiety
✅ Completed

🔍 Search 5/5: actionable steps to break patterns of fear and inconsistency
✅ Completed



In [57]:
# After collecting search results, synthesize them
SYNTHESIS_INSTRUCTIONS = "You are given multiple research summaries about overcoming fear and building consistency. \
Combine them into ONE powerful, cohesive motivational message. Keep it under 250 words. \
Make it personal, direct, and actionable. End with one clear next step."

synthesis_agent = Agent(
    name="SynthesisAgent",
    instructions=SYNTHESIS_INSTRUCTIONS,
    model="gpt-4o-mini"
)

# Combine all summaries into one input
combined_summaries = "\n\n".join([r['summary'] for r in search_results])

synthesis_result = await Runner.run(
    synthesis_agent, 
    f"Synthesize these research findings into one motivational message:\n\n{combined_summaries}"
)

# Simple HTML with just the final message
html_content = f"""
<h2>Your Message</h2>
<hr>
<div style='line-height: 1.8; font-size: 16px;'>
{synthesis_result.final_output}
</div>
<hr>
<p style='text-align: center; font-style: italic;'>You've got this.</p>
"""

In [58]:
# Define the regular function first
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    
    from_email = Email("vickyvenkata295@gmail.com")
    to_email = To("vjannatha@gmail.com")
    
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    
    response = sg.client.mail.send.post(request_body=mail)
    
    print(response.status_code)
    
    return {"status": "success", "status_code": response.status_code}


html_content = format_results_as_html(search_results)

try:
    email_result = send_email(  # ← Call the original function
        subject="Your Research Search Plan - Overcoming Fear & Consistency",
        html_body=html_content
    )
    print("✅ Email sent! Result:", email_result)
except Exception as e:
    print("❌ Send failed:", e)

202
✅ Email sent! Result: {'status': 'success', 'status_code': 202}
